## Exercício 2: word2vec

Caso você fique perdido para essa prática, algumas referências que podem ser úteis:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy array: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh plots: <http://bokeh.pydata.org/en/latest/>
* scikit-learn (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial para processar xml usando o `lxml`: <http://lxml.de/tutorial.html>

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download o dataset da TED

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download o dataset: pode demorar um pouco devido ao tamanho 75MB 
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Pré-processamento

Nesta parte, tentamos limpar um pouco as legendas, de modo que recebemos apenas sentenças. A seguinte substring mostra exemplos do que estamos tentando nos livrar. Como é difícil definir precisamente o que queremos nos livrar, usaremos apenas algumas heurísticas simples.

In [ ]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

Vamos começar removendo todas as strings entre parênteses usando um regex:

In [6]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

Podemos verificar o mesmo local no texto agora está limpo da seguinte forma. Não nos preocuparemos com os espaços irregulares, pois depois dividiremos o texto em sentenças e, em seguida, torná-lo-emos em token.

In [7]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Agora, vamos tentar remover os nomes dos alto-falantes que ocorrem no início de uma linha, excluindo partes do formato "`<até 20 caracteres>:`", conforme mostrado neste exemplo. Claro, isso é uma heurística imperfeita.

In [8]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Descomente se você precisar salvar alguma RAM: essas strings têm cerca de 50MB.
# del input_text, input_text_noparens

# Vamos ver os primeiros:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Agora que temos frases, estamos prontos para transformar cada uma delas em palavras. Essa tokenização é imperfeita, claro. Por exemplo, quantos tokens são "não" e onde / como dividimos? Nós vamos tomar a abordagem ingênua mais simples de dividir em espaços. Antes de dividir, removemos caracteres não alfanuméricos, como pontuação. Você pode considerar a seguinte pergunta: por que substituímos esses caracteres por espaços em vez de excluí-los? Pense em um caso em que isso produz uma resposta diferente.

In [9]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Duas amostras de sentenças processadas:

In [10]:
len(sentences_ted)

266694

In [11]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Frequência de palavras

Se você armazenar as contagens das 1000 principais palavras em uma lista chamada `counts_ted_top1000`, o código abaixo irá traçar o histograma solicitado no writeup.

In [12]:
# ...

Distribuição de plotagem de top-1000 palavras

In [13]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

NameError: name 'counts_ted_top1000' is not defined

### Part 3: Treinando o Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# ...

### Part 4: Ted Representações

Encontrando palavras semelhantes: (veja gensim docs para mais funcionalidades de `most_similar`)

In [14]:
model_ted.most_similar("man")

NameError: name 'model_ted' is not defined

In [15]:
model_ted.most_similar("computer")

NameError: name 'model_ted' is not defined

In [16]:
# ...

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [ ]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [ ]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [ ]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [ ]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [ ]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [ ]:
# ...

#### t-SNE visualization

In [ ]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)